In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from ncps.wirings import AutoNCP
from ncps.tf import CfC
import pickle

# Define image parameters
img_width, img_height = 64, 64

# Load and preprocess image data
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = Image.open(img_path)
            img = img.convert("RGB")
            img = img.resize((img_width, img_height), Image.BILINEAR)
            img_array = np.array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

base_folder = os.path.join('../..', 'Augmented_Dataset')

# Load images from "train" folder
train_folder = os.path.join(base_folder, 'train')
train_images, train_labels = load_images_from_folder(train_folder)

# Load images from "valid" folder
valid_folder = os.path.join(base_folder, 'valid')
valid_images, valid_labels = load_images_from_folder(valid_folder)

# Convert string labels to integer class indices
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)

# Define a custom callback to save the model's weights based on validation accuracy
class SaveBestWeights(tf.keras.callbacks.Callback):
    def __init__(self, filepath, monitor='val_accuracy', save_weights_only=True, mode='max', verbose=1):
        super(SaveBestWeights, self).__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.save_weights_only = save_weights_only
        self.mode = mode
        self.verbose = verbose
        self.best = -np.Inf if mode == 'max' else np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            if self.verbose > 0:
                print("Warning: Can't save best weights. No {} found.".format(self.monitor))
        else:
            if self.mode == 'max' and current > self.best:
                if self.verbose > 0:
                    print("Validation {} improved from {:.5f} to {:.5f}, saving weights.".format(self.monitor, self.best, current))
                self.best = current
                Mobile_NetModel = tf.keras.Model(inputs=self.model.input, outputs=self.model.layers[-3].output)
                Mobile_NetModel.save("Hybrid_lstm_CNN_weights.h5")

                # Save the trainable parameters of the LSTM layer as NumPy arrays using pickle
                ncp_weights = self.model.layers[-2].get_weights()
                pickle.dump(ncp_weights, open('Hybrid_lstm_weights.pkl', 'wb'))

                # Save the weights of the last dense layer
                last_dense_layer_weights = self.model.layers[-1].get_weights()
                pickle.dump(last_dense_layer_weights, open('Hybrid_lstm_last_dense_layer_weights.pkl', 'wb'))

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))

# CfC Model
lstm = LSTM(64)

# Build a new model on top of the base MobileNetV2 model
model = Sequential([
    base_model,
    Reshape((4, 1280)),
    lstm,
    Dense(11, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the temporary filepath to save the best weights
best_weights_filepath = 'best_weights.h5'

# Define the callback to save the best weights based on validation accuracy
save_best_weights_callback = SaveBestWeights(filepath=best_weights_filepath)

# Train the model with the callback
model.fit(train_images, train_labels_encoded, epochs=50, batch_size=16, validation_data=(valid_images, valid_labels_encoded), callbacks=[save_best_weights_callback])
